# Multi-Agent Therapeutic Conversation with Cognitive Action Probes

This notebook explores therapeutic conversations between two AI agents (therapist and client) using cognitive action probes to analyze their thinking patterns.

**Features:**
- ✅ Therapist-Client conversation simulation
- ✅ Real-time cognitive action detection using probes
- ✅ Multiple conversation scenarios to test
- ✅ 6-10 turn conversations
- ✅ Detailed probe analysis for both agents

**Requirements:**
- Google Colab with GPU (T4 or better)
- ~10 GB VRAM
- Runtime: ~15-20 minutes total

## 1️⃣ Check GPU and Setup

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print("\n" + "="*60)
print("GPU INFORMATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected! This will be very slow on CPU.")
print("="*60)

## 2️⃣ Clone Repository and Install Dependencies

In [ ]:
import os
import sys

# Clone the repository
repo_url = "https://github.com/ChuloIva/brije.git"
repo_name = "brije"

if not os.path.exists(repo_name):
    print("📥 Cloning Brije repository...")
    !git clone {repo_url}
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")
    print("🔄 Pulling latest changes...")
    !cd {repo_name} && git pull

# Change to repo directory
os.chdir(repo_name)
print(f"\n📁 Current directory: {os.getcwd()}")

In [ ]:
# Install dependencies
print("📦 Installing dependencies...\n")

# Core dependencies for the notebook
print("Installing core packages...")
!pip install -q torch transformers h5py scikit-learn tqdm matplotlib seaborn

# Dependencies for liminal_backrooms
print("\nInstalling liminal_backrooms dependencies...")
!pip install -q python-dotenv requests Pillow

# Optional API clients (only needed if you want to use other models later)
# These won't be used for Gemma with probes, but prevents import errors
print("\nInstalling optional API clients (prevents import errors)...")
!pip install -q anthropic openai replicate together

# Clone and install nnsight
nnsight_dir = "third_party/nnsight"
nnsight_repo = "https://github.com/ndif-team/nnsight"

print("\n📦 Setting up nnsight...")
if not os.path.exists(nnsight_dir) or not os.listdir(nnsight_dir):
    print("   Cloning nnsight repository...")
    os.makedirs("third_party", exist_ok=True)
    !git clone {nnsight_repo} {nnsight_dir}
    print("   ✅ nnsight repository cloned")
else:
    print("   ✅ nnsight repository already exists")

# Install nnsight
print("   Installing nnsight...")
!pip install -q -e {nnsight_dir}

print("\n✅ All dependencies installed!")
print("\n📋 Installed packages:")
print("   • PyTorch + Transformers (for Gemma 3 4B)")
print("   • nnsight (for activation extraction)")
print("   • scikit-learn (for probe models)")
print("   • python-dotenv, requests, Pillow (liminal_backrooms)")
print("   • matplotlib, seaborn (visualization)")
print("   • anthropic, openai, replicate, together (optional - prevents import errors)")

## 3️⃣ Setup Environment (No API Keys Required!)

**Note:** When using Gemma 3 4B with probes, the model runs **locally** on your GPU. No API keys or internet required!

This cell creates a minimal .env file to prevent import errors.

In [ ]:
import os

# Create a minimal .env file (not needed for Gemma, but prevents import errors)
with open('.env', 'w') as f:
    f.write("# Gemma 3 4B runs locally - no API keys needed\n")

print("✅ Environment setup complete!")
print("\n💡 Important: Gemma 3 4B with probes runs entirely on your GPU.")
print("   • No API calls")
print("   • No API keys") 
print("   • No internet required")
print("   • 100% local inference with real-time probe analysis!")

## 4️⃣ Download Pre-trained Probes (Optional but Recommended)

If you have pre-trained probes, upload them to `data/probes_binary/`. Otherwise, the system will run without probe visualization.

In [2]:
# Check if probes exist
import glob
import os
probe_dirs = glob.glob('data/probes_binary/layer_*')

if probe_dirs:
    print("✅ Found pre-trained probes:")
    for probe_dir in sorted(probe_dirs)[:5]:
        layer_num = os.path.basename(probe_dir).replace('layer_', '')
        probe_files = glob.glob(f"{probe_dir}/probe_*.pth")
        print(f"   Layer {layer_num}: {len(probe_files)} probes")
    if len(probe_dirs) > 5:
        print(f"   ... and {len(probe_dirs) - 5} more layers")
    print("\n🎯 Probes will be used for cognitive action detection!")
else:
    print("⚠️  No pre-trained probes found in data/probes_binary/")
    print("\n💡 Options:")
    print("   1. Upload pre-trained probes to data/probes_binary/layer_XX/")
    print("   2. Train probes using Brije_Full_Pipeline_Colab.ipynb")
    print("   3. Continue without probes (conversations will still work)")
    print("\n⚠️  Continuing without probe visualization...")

✅ Found pre-trained probes:
   Layer 21: 45 probes
   Layer 22: 45 probes
   Layer 23: 45 probes
   Layer 24: 45 probes
   Layer 25: 45 probes
   ... and 5 more layers

🎯 Probes will be used for cognitive action detection!


## 5️⃣ Add Therapeutic Conversation System Prompts

We'll add a new system prompt pair for therapist-client conversations.

In [3]:
# Add therapeutic conversation prompts to config
import sys
from pathlib import Path

# Add liminal_backrooms to path
sys.path.insert(0, str(Path.cwd() / "third_party" / "liminal_backrooms"))

from third_party.liminal_backrooms.config import SYSTEM_PROMPT_PAIRS

# Add therapeutic conversation prompt
SYSTEM_PROMPT_PAIRS["Therapeutic Session - Therapist and Client"] = {
    "AI_1": """You are a skilled therapist conducting a therapy session. Your role is to:
- Listen actively and empathetically to the client
- Ask thoughtful, open-ended questions to help the client explore their feelings
- Notice patterns in the client's thinking and behavior
- Help the client reframe negative thoughts and consider new perspectives
- Validate emotions while gently challenging unhelpful beliefs
- Guide the client toward insights and actionable strategies
- Use techniques like cognitive reframing, perspective-taking, and emotional awareness

Be warm, professional, and focused on helping the client gain clarity and develop coping strategies.""",
    
    "AI_2": """You are a client in a therapy session. You are seeking help with various challenges such as:
- Anxiety, stress, or overwhelming emotions
- Relationship difficulties
- Work-related stress or burnout
- Self-doubt or negative self-talk
- Difficulty making decisions
- Processing difficult experiences

Express your thoughts and feelings authentically. Sometimes you're uncertain, sometimes you have insights, sometimes you're confused. Be open to exploring your thinking patterns and considering new perspectives the therapist offers."""
}

print("✅ Added therapeutic conversation system prompts!")
print("\nAvailable conversation types:")
for i, prompt_name in enumerate(SYSTEM_PROMPT_PAIRS.keys(), 1):
    print(f"   {i}. {prompt_name}")

✅ Added therapeutic conversation system prompts!

Available conversation types:
   1. Backrooms
   2. ASCII Art
   3. Image Model Collaboration
   4. Cognitive Roles - Analyst vs Creative
   5. Cognitive Roles - Skeptic vs Optimist
   6. Cognitive Roles - Metacognitive Explorer
   7. Therapeutic Session - Therapist and Client


## 6️⃣ Define Test Scenarios

Different opening statements from the client to explore various therapeutic situations.

In [5]:
# Define therapeutic conversation scenarios
THERAPEUTIC_SCENARIOS = [
    {
        "name": "Anxiety and Overthinking",
        "opening": "I've been feeling really anxious lately. My mind just keeps racing with all these worst-case scenarios, and I can't seem to shut it off. It's affecting my sleep and my work."
    },
    {
        "name": "Relationship Conflict",
        "opening": "I had another argument with my partner yesterday. It feels like we keep having the same fight over and over. I don't know if we're just not compatible or if I'm doing something wrong."
    },
    # {
    #     "name": "Imposter Syndrome",
    #     "opening": "I got promoted at work, but instead of feeling excited, I just feel terrified. I keep thinking they made a mistake choosing me, and everyone's going to realize I'm not actually good enough."
    # },
    # {
    #     "name": "Burnout and Exhaustion",
    #     "opening": "I'm so tired all the time. I used to love my job, but now just thinking about work makes me feel drained. I don't know if I should push through or if something needs to change."
    # },
    # {
    #     "name": "Difficult Decision",
    #     "opening": "I've been offered a job in another city. It's a great opportunity, but it would mean leaving my family and friends. I keep going back and forth, and I can't decide what's right."
    # },
    # {
    #     "name": "Grief and Loss",
    #     "opening": "It's been six months since I lost my dad, and people keep telling me I should be 'moving on.' But some days it hits me just as hard as it did in the beginning."
    # }
]

print("📋 Therapeutic Scenarios:")
print("="*60)
for i, scenario in enumerate(THERAPEUTIC_SCENARIOS, 1):
    print(f"\n{i}. {scenario['name']}")
    print(f"   Opening: {scenario['opening'][:80]}...")
print("\n" + "="*60)

📋 Therapeutic Scenarios:

1. Anxiety and Overthinking
   Opening: I've been feeling really anxious lately. My mind just keeps racing with all thes...

2. Relationship Conflict
   Opening: I had another argument with my partner yesterday. It feels like we keep having t...



## 7️⃣ Run Multi-Agent Therapeutic Conversations

This will run conversations for each scenario with cognitive action detection.

In [6]:
from third_party.liminal_backrooms.main import ai_turn
from third_party.liminal_backrooms.config import AI_MODELS
import json
import time
from datetime import datetime

def run_therapeutic_conversation(scenario, num_turns=3):
    """
    Run a therapeutic conversation for a given scenario.
    
    Args:
        scenario: Dict with 'name' and 'opening' keys
        num_turns: Number of conversation turns (default 8 = 4 exchanges)
    """
    print("\n" + "="*80)
    print(f"SCENARIO: {scenario['name']}")
    print("="*80)
    print(f"\nClient's opening: {scenario['opening']}")
    print("\n" + "-"*80)
    
    # Configuration
    therapist_model = "Gemma 3 4B (with Probes)"
    client_model = "Gemma 3 4B (with Probes)"
    
    # Get system prompts
    prompt_pair = "Therapeutic Session - Therapist and Client"
    therapist_prompt = SYSTEM_PROMPT_PAIRS[prompt_pair]["AI_1"]
    client_prompt = SYSTEM_PROMPT_PAIRS[prompt_pair]["AI_2"]
    
    # Initialize conversation with client's opening
    conversation = [
        {
            "role": "user",
            "content": scenario['opening']
        }
    ]
    
    # Run conversation turns
    for turn in range(num_turns):
        # Alternate between therapist (AI-1) and client (AI-2)
        is_therapist = (turn % 2 == 0)
        ai_name = "Therapist" if is_therapist else "Client"
        model = therapist_model if is_therapist else client_model
        system_prompt = therapist_prompt if is_therapist else client_prompt
        
        print(f"\n{'='*80}")
        print(f"TURN {turn + 1}: {ai_name}")
        print("="*80)
        
        # Run the AI turn
        conversation = ai_turn(
            ai_name=ai_name,
            conversation=conversation,
            model=model,
            system_prompt=system_prompt,
            gui=None
        )
        
        # Display the latest response
        latest = conversation[-1]
        print(f"\n{ai_name}:")
        print("-" * 40)
        print(latest.get('content', ''))
        
        # Display cognitive action predictions if available
        if 'predictions' in latest:
            print("\n" + "-" * 40)
            print("🧠 COGNITIVE ACTIONS DETECTED:")
            print("-" * 40)
            predictions = latest['predictions']
            for i, pred in enumerate(predictions[:5], 1):  # Show top 5
                action = pred.get('action', 'Unknown')
                confidence = pred.get('confidence', 0.0)
                is_active = pred.get('is_active', False)
                marker = "✓" if is_active else "○"
                print(f"  {marker} {i}. {action:35s} {confidence:6.1%}  {'[ACTIVE]' if is_active else ''}")
        
        # Small delay between turns
        time.sleep(1)
    
    # Save conversation
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = Path('output/therapeutic_conversations')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / f"{scenario['name'].replace(' ', '_')}_{timestamp}.json"
    with open(output_file, 'w') as f:
        json.dump({
            'scenario': scenario,
            'conversation': conversation,
            'metadata': {
                'therapist_model': therapist_model,
                'client_model': client_model,
                'num_turns': num_turns,
                'timestamp': timestamp
            }
        }, f, indent=2, default=str)
    
    print("\n" + "="*80)
    print(f"✅ Conversation saved to: {output_file}")
    print("="*80)
    
    return conversation, output_file

print("✅ Therapeutic conversation function ready!")

ModuleNotFoundError: No module named '_tkinter'

## 8️⃣ Run All Scenarios

Run therapeutic conversations for all scenarios (6-10 turns each).

In [ ]:
# Configure number of turns
NUM_TURNS = 8  # 8 turns = 4 therapist responses + 4 client responses

print(f"Running {len(THERAPEUTIC_SCENARIOS)} therapeutic scenarios...")
print(f"Each conversation will have {NUM_TURNS} turns ({NUM_TURNS//2} exchanges)\n")

results = []
start_time = time.time()

for i, scenario in enumerate(THERAPEUTIC_SCENARIOS, 1):
    print(f"\n{'#'*80}")
    print(f"RUNNING SCENARIO {i}/{len(THERAPEUTIC_SCENARIOS)}")
    print(f"{'#'*80}")
    
    try:
        conversation, output_file = run_therapeutic_conversation(scenario, num_turns=NUM_TURNS)
        results.append({
            'scenario': scenario['name'],
            'success': True,
            'output_file': str(output_file)
        })
    except Exception as e:
        print(f"\n❌ Error in scenario '{scenario['name']}': {str(e)}")
        results.append({
            'scenario': scenario['name'],
            'success': False,
            'error': str(e)
        })
    
    # Add delay between scenarios
    if i < len(THERAPEUTIC_SCENARIOS):
        print("\n⏸️  Pausing 5 seconds before next scenario...")
        time.sleep(5)

elapsed_time = time.time() - start_time

# Summary
print("\n\n" + "#"*80)
print("ALL SCENARIOS COMPLETE")
print("#"*80)
print(f"\nTotal time: {elapsed_time/60:.1f} minutes")
print(f"Successful: {sum(1 for r in results if r['success'])}/{len(results)}")

print("\n📊 Results Summary:")
for result in results:
    status = "✅" if result['success'] else "❌"
    print(f"{status} {result['scenario']}")
    if result['success']:
        print(f"   Output: {result['output_file']}")
    else:
        print(f"   Error: {result['error']}")

print("\n" + "#"*80)

## 9️⃣ Analyze Cognitive Actions Across Conversations

Analyze which cognitive actions appear most frequently in therapist vs client responses.

In [ ]:
import json
from collections import defaultdict, Counter
from pathlib import Path

# Load all saved conversations
output_dir = Path('output/therapeutic_conversations')
conversation_files = list(output_dir.glob('*.json'))

if not conversation_files:
    print("⚠️  No conversation files found. Run the conversations first!")
else:
    print(f"📊 Analyzing {len(conversation_files)} conversations...\n")
    
    therapist_actions = Counter()
    client_actions = Counter()
    
    for conv_file in conversation_files:
        with open(conv_file, 'r') as f:
            data = json.load(f)
        
        conversation = data['conversation']
        
        # Analyze each turn
        for i, turn in enumerate(conversation):
            if 'predictions' in turn:
                # Therapist speaks on even turns (0, 2, 4...)
                is_therapist = (i % 2 == 1)  # Skip initial user message
                
                for pred in turn['predictions']:
                    if pred.get('is_active', False):
                        action = pred['action']
                        if is_therapist:
                            therapist_actions[action] += 1
                        else:
                            client_actions[action] += 1
    
    print("="*80)
    print("TOP COGNITIVE ACTIONS: THERAPIST")
    print("="*80)
    for action, count in therapist_actions.most_common(15):
        bar = "█" * (count // 2)
        print(f"{action:35s} {count:3d} {bar}")
    
    print("\n" + "="*80)
    print("TOP COGNITIVE ACTIONS: CLIENT")
    print("="*80)
    for action, count in client_actions.most_common(15):
        bar = "█" * (count // 2)
        print(f"{action:35s} {count:3d} {bar}")
    
    # Find unique actions
    therapist_only = set(therapist_actions.keys()) - set(client_actions.keys())
    client_only = set(client_actions.keys()) - set(therapist_actions.keys())
    
    if therapist_only:
        print("\n" + "="*80)
        print("COGNITIVE ACTIONS UNIQUE TO THERAPIST")
        print("="*80)
        for action in sorted(therapist_only):
            print(f"  • {action}")
    
    if client_only:
        print("\n" + "="*80)
        print("COGNITIVE ACTIONS UNIQUE TO CLIENT")
        print("="*80)
        for action in sorted(client_only):
            print(f"  • {action}")
    
    # Save analysis
    analysis = {
        'therapist_actions': dict(therapist_actions.most_common(20)),
        'client_actions': dict(client_actions.most_common(20)),
        'therapist_only': list(therapist_only),
        'client_only': list(client_only)
    }
    
    with open(output_dir / 'cognitive_action_analysis.json', 'w') as f:
        json.dump(analysis, f, indent=2)
    
    print("\n✅ Analysis saved to: output/therapeutic_conversations/cognitive_action_analysis.json")

## 🔟 Visualize Cognitive Action Patterns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if therapist_actions and client_actions:
    # Get top actions from both
    top_therapist = dict(therapist_actions.most_common(10))
    top_client = dict(client_actions.most_common(10))
    
    # Combine and get unique actions
    all_actions = set(list(top_therapist.keys()) + list(top_client.keys()))
    
    # Create comparison data
    actions = sorted(all_actions)
    therapist_counts = [top_therapist.get(a, 0) for a in actions]
    client_counts = [top_client.get(a, 0) for a in actions]
    
    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Comparison bar chart
    x = np.arange(len(actions))
    width = 0.35
    
    axes[0].barh(x - width/2, therapist_counts, width, label='Therapist', color='steelblue', alpha=0.8)
    axes[0].barh(x + width/2, client_counts, width, label='Client', color='coral', alpha=0.8)
    axes[0].set_yticks(x)
    axes[0].set_yticklabels(actions, fontsize=9)
    axes[0].set_xlabel('Frequency', fontsize=11)
    axes[0].set_title('Cognitive Action Comparison:\nTherapist vs Client', fontsize=13, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='x')
    axes[0].invert_yaxis()
    
    # Plot 2: Top 5 for each role
    top5_therapist = dict(therapist_actions.most_common(5))
    top5_client = dict(client_actions.most_common(5))
    
    y_pos = np.arange(len(top5_therapist) + len(top5_client) + 1)
    colors = ['steelblue'] * len(top5_therapist) + ['white'] + ['coral'] * len(top5_client)
    heights = list(top5_therapist.values()) + [0] + list(top5_client.values())
    labels = list(top5_therapist.keys()) + [''] + list(top5_client.keys())
    
    axes[1].barh(y_pos, heights, color=colors, alpha=0.8, edgecolor='black')
    axes[1].set_yticks(y_pos)
    axes[1].set_yticklabels(labels, fontsize=9)
    axes[1].set_xlabel('Frequency', fontsize=11)
    axes[1].set_title('Top 5 Cognitive Actions\nby Role', fontsize=13, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')
    axes[1].invert_yaxis()
    
    # Add labels
    axes[1].text(0.02, 0.95, 'Therapist', transform=axes[1].transAxes,
                fontsize=11, fontweight='bold', color='steelblue',
                verticalalignment='top')
    axes[1].text(0.02, 0.35, 'Client', transform=axes[1].transAxes,
                fontsize=11, fontweight='bold', color='coral',
                verticalalignment='top')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'cognitive_action_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Visualization saved to: output/therapeutic_conversations/cognitive_action_comparison.png")
else:
    print("⚠️  No cognitive action data available. Make sure probes are loaded!")

## 1️⃣1️⃣ Run a Custom Scenario

Create and run your own therapeutic conversation scenario.

In [ ]:
# Define your custom scenario
custom_scenario = {
    "name": "Custom Scenario",
    "opening": "I feel like I'm stuck in a loop. Every time I try to make a change, I end up back where I started. I don't know what I'm doing wrong."
}

# Customize the number of turns (6-10 recommended)
custom_num_turns = 10

print("Running custom therapeutic conversation...\n")
conversation, output_file = run_therapeutic_conversation(custom_scenario, num_turns=custom_num_turns)

print(f"\n✅ Custom conversation complete!")
print(f"📁 Saved to: {output_file}")

## 1️⃣2️⃣ Summary and Next Steps

This notebook explored therapeutic conversations with cognitive action detection:

**What we learned:**
- How therapists and clients use different cognitive actions
- Which cognitive patterns emerge in different scenarios
- How probe-based analysis can reveal thinking patterns

**Next steps:**
1. Analyze the saved conversations in `output/therapeutic_conversations/`
2. Compare cognitive actions across different scenarios
3. Experiment with different system prompts
4. Try longer conversations (10-15 turns)
5. Analyze specific therapeutic techniques used

**Files generated:**
- Individual conversation JSONs with full probe data
- `cognitive_action_analysis.json` - Aggregate statistics
- `cognitive_action_comparison.png` - Visualization